# Simulation Design

_Control Complexity with Modular DGPs_

In [ ]:
# Re-import necessary libraries after code execution reset
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.lines import Line2D

# Use a serif font and increase font sizes for readability
mpl.rcParams.update({
    "font.family": "serif",
    "font.size": 14,
    "axes.titlesize": 16,
    "axes.labelsize": 14,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "legend.fontsize": 12,
    "axes.grid": True,
    "grid.linestyle": "--",
    "grid.alpha": 0.6,
    "axes.spines.top": False,
    "axes.spines.right": False,
})

In [ ]:
def simulate_rl_panel(stage: int, seed: int = 42) -> pd.DataFrame:
    """
    Simulates panel data under three staged DGPs for evaluating RL donor selection.

    Parameters:
    - stage (int): Simulation stage (1 to 3)
        1 = Static DGP, no regime change
        2 = Fixed A parameters with Gaussian bump for shock
        3 = Fully randomized parameters with Gaussian bump for A
    - seed (int): Random seed for reproducibility

    Returns:
    - pd.DataFrame: Simulated panel data
    """
    np.random.seed(seed)
    units = [chr(65 + i) for i in range(10)]  # Units A–J
    periods = pd.date_range(start="1960-03-01", end="2024-12-01", freq="3MS") # 260 obs
    T = len(periods)

    # Parameters
    p, q = 0.98, 0.6
    alpha = 0.09
    beta = 2
    sigmay = 1
    sigmax = 0.5

    data = []

    for unit in units:
        if stage == 2 and unit == "A":
            rho_i = 0.8
            gamma_i = 1.4
        else:
            rho_i = np.random.uniform(0.4, 0.9)
            gamma_i = np.random.uniform(0.5, 1.5)

        y = np.zeros(T)
        delta = np.zeros(T)
        x = np.zeros(T)

        signal = np.sin(np.linspace(0, 6 * np.pi, T))

        if stage == 1:
            x = np.random.normal(0, sigmax, T)
            delta[:] = 0

        elif stage in [2, 3]:
            if unit == "A":
                center_time = pd.Timestamp("2015-06-01")  # midpoint of 2010–2019
                center_idx = np.argmin(np.abs(periods - center_time))
                width = 8  # 8 quarters is 2 years, on each side is roughly 2013–2017
                base_level = -4
                scale = 10

                bump = base_level + scale * np.exp(-((np.arange(T) - center_idx) ** 2) / (2 * width ** 2))
            else:
                bump = np.zeros(T)

            x = gamma_i * (signal + bump) + np.random.normal(0, sigmax, T)

        for t in range(1, T):
            X = x[t - 1]
            y_lag = y[t - 1]

            if stage == 1:
                delta[t] = 0
            else:
                if delta[t - 1] == 0:
                    prob = 1 - (p - alpha * X)
                else:
                    prob = q
                prob = np.clip(prob, 0, 1)
                delta[t] = np.random.binomial(1, prob)

            epsilon = np.random.normal(0, sigmay)
            y[t] = rho_i * y[t - 1] + beta * delta[t] + epsilon

            data.append({
                'unit': unit,
                'period': periods[t],
                'y': y[t],
                'y_lag': y_lag,
                'delta': delta[t],
                'x': x[t],
                'x_lag': x[t - 1],
                'rho_i': rho_i,
                'gamma_i': gamma_i,
                'stage': stage
            })

    df = pd.DataFrame(data)
    df['period'] = pd.to_datetime(df['period'])  # ensure datetime format
    return df

## Plot simulated data for Target A

In [ ]:
stage = 3  
n_simulations = 100
fig, ax = plt.subplots(figsize=(10, 4))

for seed in range(n_simulations):
    df_sim = simulate_rl_panel(stage=stage, seed=seed)
    df_sim.set_index('period', inplace=True)
    df_A = df_sim[df_sim['unit'] == 'A']
    ax.plot(df_A.index, df_A['x'], alpha=0.2, color='#0072B2', linewidth=1)
    ax.plot(df_A.index, df_A['y'], alpha=0.2, color='#D55E00', linewidth=1)

# Add titles and labels
#ax.set_title(f'Simulated Trajectories for Unit A over {n_simulations} Simulations')
ax.set_xlim(df_A.index.min(), df_A.index.max())
ax.set_xlabel('Period')
ax.set_ylabel('Value')

# Custom legend using Line2D
legend_elements = [
    Line2D([0], [0], color='#0072B2', lw=2, label='x (feature)'),
    Line2D([0], [0], color='#D55E00', lw=2, label='y (outcome)')
]
ax.legend(handles=legend_elements, loc='upper left', frameon=False)

# Tighter layout and export-ready
plt.tight_layout()
plt.savefig(f"results/regime/stage{stage}/figures/stage{stage}_simulated_data.pdf", format="pdf", bbox_inches="tight", dpi=300)
plt.show()

# Monte Carlo Loop Design

In [ ]:
import os
from OptimalPanel.optimizer import OptimalBundleRL
from tqdm import tqdm

# ---- Configurable Parameters ----
n_simulations = 100
total_epochs = 500
stage = 3  # Simulation stage to use
base_dir = f"results/regime/stage{stage}"

model_dir = os.path.join(base_dir, "models")
data_dir = os.path.join(base_dir, "data")
os.makedirs(model_dir, exist_ok=True)
os.makedirs(data_dir, exist_ok=True)

forecast_times = list(pd.date_range(start="2012-03-01", end="2018-12-01", freq="3MS"))
test_start = pd.to_datetime("1970-03-01")
test_end = pd.to_datetime("1975-12-01")

# ---- Monte Carlo Loop ----
for sim in tqdm(range(n_simulations), desc="Running Monte Carlo"):
    
    # Simulate Data
    df_sim = simulate_rl_panel(stage=stage, seed=sim)

    # Save DataFrame
    df_path = os.path.join(data_dir, f"df_sim_{sim}.parquet")
    df_sim.to_parquet(df_path, index=False)

    # Initialize RL Framework
    rl = OptimalBundleRL(
        df=df_sim,
        unit_col='unit',
        time_col='period',
        target_col='y',
        feature_cols=['y_lag', 'x_lag'],
        target_unit='A',
        forecast_times=forecast_times
    )

    # Compute Similarities
    try:
        rl.compute_similarities(
            test_start=test_start,
            test_end=test_end
        )
    except ValueError as e:
        print(f"❌ Skipping simulation {sim}: {e}")
        continue

    # Step 4: Train RL and Save Model Output
    model_path = os.path.join(model_dir, f"results_sim_{sim}.pkl")
    rl.train(
        n_epochs=total_epochs,
        ar_exo='n',  # No exogenous variable for AR(1) model
        save=True,
        save_path=model_path
    )